In [150]:
import pandas as pd
import numpy as np
from datetime import date

In [170]:
off_train = pd.read_csv('/content/drive/MyDrive/5012project/ccf_offline_stage1_train.csv',header = None, low_memory= False)
off_train.columns = ['user_id','merchant_id','coupon_id','discount_rate','distance','date_received','date']
#off_train.head()
#2050 coupon_id. date_received:20160701~20160731, 76309 users(76307 in trainset, 35965 in online_trainset), 1559 merchants(1558 in trainset)

off_test = pd.read_csv('/content/drive/MyDrive/5012project/ccf_offline_stage1_test_revised.csv',low_memory=False)
off_test.columns = ['user_id','merchant_id','coupon_id','discount_rate','distance','date_received']
#off_train.head()
#11429826 record(872357 with coupon_id),762858 user(267448 in off_train)

on_train = pd.read_csv('/content/drive/MyDrive/5012project/ccf_online_stage1_train.csv',header= None,low_memory=False)
on_train.columns = ['user_id','merchant_id','action','coupon_id','discount_rate','date_received','date']
#off_train.head()

In [152]:
off_train.info()
#off_test.head()
#on_train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754885 entries, 0 to 1754884
Data columns (total 7 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   user_id        object
 1   merchant_id    object
 2   coupon_id      object
 3   discount_rate  object
 4   distance       object
 5   date_received  object
 6   date           object
dtypes: object(7)
memory usage: 93.7+ MB


In [171]:
dataset3 = off_test
feature3 = off_train[((off_train.date>='20160315')&(off_train.date<='20160630'))|((off_train.date=='null')&(off_train.date_received>='20160315')&(off_train.date_received<='20160630'))]
dataset2 = off_train[(off_train.date_received>='20160515')&(off_train.date_received<='20160615')]
feature2 = off_train[(off_train.date>='20160201')&(off_train.date<='20160514')|((off_train.date=='null')&(off_train.date_received>='20160201')&(off_train.date_received<='20160514'))]
dataset1 = off_train[(off_train.date_received>='20160414')&(off_train.date_received<='20160514')]
feature1 = off_train[(off_train.date>='20160101')&(off_train.date<='20160413')|((off_train.date=='null')&(off_train.date_received>='20160101')&(off_train.date_received<='20160413'))]

In [155]:
dataset3.head()

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received
0,4129537,450,9983,30:5,1.0,20160712
1,6949378,1300,3429,30:5,NaN,20160706
2,2166529,7113,6928,200:20,5.0,20160727
3,2166529,7113,1808,100:10,5.0,20160727
4,6172162,7605,6500,30:1,2.0,20160708


In [ ]:
#print(feature3.count())

user_id          554179
merchant_id      554179
coupon_id         55158
discount_rate     55158
distance         548054
date_received     55158
date             554179
dtype: int64


In [156]:
############# coupon related feature   #############
def calc_discount_rate(s):
    s =str(s)
    s = s.split(':')
    if len(s)==1:
        return float(s[0])
    else:
        return 1.0-float(s[1])/float(s[0])

def get_discount_man(s):
    s =str(s)
    s = s.split(':')
    if len(s)==1:
        return 'null'
    else:
        return int(s[0])
        
def get_discount_jian(s):
    s =str(s)
    s = s.split(':')
    if len(s)==1:
        return 'null'
    else:
        return int(s[1])

def is_man_jian(s):
    s =str(s)
    s = s.split(':')
    if len(s)==1:
        return 0
    else:
        return 1

In [157]:
#dataset3
dataset3['day_of_week'] = dataset3.date_received.astype('str').apply(lambda x:date(int(x[0:4]),int(x[4:6]),int(x[6:8])).weekday()+1)
dataset3['day_of_month'] = dataset3.date_received.astype('str').apply(lambda x:int(x[6:8]))
dataset3['days_distance'] = dataset3.date_received.astype('str').apply(lambda x:(date(int(x[0:4]),int(x[4:6]),int(x[6:8]))-date(2016,6,30)).days)
dataset3['discount_man'] = dataset3.discount_rate.apply(get_discount_man)
dataset3['discount_jian'] = dataset3.discount_rate.apply(get_discount_jian)
dataset3['is_man_jian'] = dataset3.discount_rate.apply(is_man_jian)
dataset3['discount_rate'] = dataset3.discount_rate.apply(calc_discount_rate)
d['coupon_id']= dataset3[['coupon_id']]
d['coupon_count'] = 1
d = d.groupby('coupon_id').agg('sum').reset_index()
dataset3 = pd.merge(dataset3,d,on='coupon_id',how='left')
dataset3.to_csv('/content/drive/MyDrive/5012project/coupon3_feature.csv',index=None)

In [163]:
dataset3.head()

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,day_of_week,day_of_month,days_distance,discount_man,discount_jian,is_man_jian,coupon_count
0,4129537,450,9983,0.833333,1.0,20160712,2,12,12,30,5,1,10.0
1,6949378,1300,3429,0.833333,NaN,20160706,3,6,6,30,5,1,2.0
2,2166529,7113,6928,0.900000,5.0,20160727,3,27,27,200,20,1,1.0
3,2166529,7113,1808,0.900000,5.0,20160727,3,27,27,100,10,1,1.0
4,6172162,7605,6500,0.966667,2.0,20160708,5,8,8,30,1,1,1.0


In [173]:
dataset2.head()

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,date,day_of_week,day_of_month,days_distance,discount_man,discount_jian,is_man_jian,coupon_count
0,1439408,4663,11002,0.866667,1,20160528,NaN,6,28,14,150,20,1,2.0
1,1439408,2632,8591,0.950000,0,20160613,NaN,1,13,30,20,1,1,2.0
2,1439408,2632,8591,0.950000,0,20160516,20160613,1,16,2,20,1,1,2.0
3,2029232,450,1532,0.833333,0,20160530,NaN,1,30,16,30,5,1,1.0
4,2029232,6459,12737,0.950000,0,20160519,NaN,4,19,5,20,1,1,1.0


In [175]:
dataset1.head()

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,date,day_of_week,day_of_month,days_distance,discount_man,discount_jian,is_man_jian,coupon_count
0,1832624,3381,7610,0.900000,0,20160429,NaN,5,29,16,200,20,1,1.0
1,163606,1569,5054,0.850000,10,20160421,NaN,4,21,8,200,30,1,NaN
2,4061024,3381,7610,0.900000,10,20160426,NaN,2,26,13,200,20,1,1.0
3,106443,450,3732,0.833333,NaN,20160429,NaN,5,29,16,30,5,1,NaN
4,114747,1569,5054,0.850000,9,20160426,NaN,2,26,13,200,30,1,NaN


In [172]:
#dataset2
dataset2['day_of_week'] = dataset2.date_received.astype('str').apply(lambda x:date(int(x[0:4]),int(x[4:6]),int(x[6:8])).weekday()+1)
dataset2['day_of_month'] = dataset2.date_received.astype('str').apply(lambda x:int(x[6:8]))
dataset2['days_distance'] = dataset2.date_received.astype('str').apply(lambda x:(date(int(x[0:4]),int(x[4:6]),int(x[6:8]))-date(2016,5,14)).days)
dataset2['discount_man'] = dataset2.discount_rate.apply(get_discount_man)
dataset2['discount_jian'] = dataset2.discount_rate.apply(get_discount_jian)
dataset2['is_man_jian'] = dataset2.discount_rate.apply(is_man_jian)
dataset2['discount_rate'] = dataset2.discount_rate.apply(calc_discount_rate)
d['coupon_id']= dataset2[['coupon_id']]
d['coupon_count'] = 1
d = d.groupby('coupon_id').agg('sum').reset_index()
dataset2 = pd.merge(dataset2,d,on='coupon_id',how='left')
dataset2.to_csv('/content/drive/MyDrive/5012project/coupon2_feature.csv',index=None)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [174]:
#dataset1
dataset1['day_of_week'] = dataset1.date_received.astype('str').apply(lambda x:date(int(x[0:4]),int(x[4:6]),int(x[6:8])).weekday()+1)
dataset1['day_of_month'] = dataset1.date_received.astype('str').apply(lambda x:int(x[6:8]))
dataset1['days_distance'] = dataset1.date_received.astype('str').apply(lambda x:(date(int(x[0:4]),int(x[4:6]),int(x[6:8]))-date(2016,4,13)).days)
dataset1['discount_man'] = dataset1.discount_rate.apply(get_discount_man)
dataset1['discount_jian'] = dataset1.discount_rate.apply(get_discount_jian)
dataset1['is_man_jian'] = dataset1.discount_rate.apply(is_man_jian)
dataset1['discount_rate'] = dataset1.discount_rate.apply(calc_discount_rate)
d['coupon_id'] = dataset1[['coupon_id']]
d['coupon_count'] = 1
d = d.groupby('coupon_id').agg('sum').reset_index()
dataset1 = pd.merge(dataset1,d,on='coupon_id',how='left')
dataset1.to_csv('/content/drive/MyDrive/5012project/coupon1_feature.csv',index=None)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead